In [434]:
from ortools.sat.python import cp_model
import time
import pandas
import numpy as np
import itertools

Initialise model

In [435]:
model = cp_model.CpModel()

Route \<generated from route scheduling\>

In [436]:
points_start = ['A', 'B']
points_holding_1 = ['C', 'D']
points_holding_2 = ['C1', 'D1']
points_destination = ['E', 'F']
points = points_start + points_holding_1 + points_holding_2 + points_destination
routes = pandas.DataFrame(999, index=points, columns=points)
routes.loc['A', 'C'], routes.loc['C', 'A'] = 5, 5
routes.loc['A', 'D'], routes.loc['D', 'A'] = 6, 6
routes.loc['B', 'C'], routes.loc['C', 'B'] = 4, 4
routes.loc['B', 'D'], routes.loc['D', 'B'] = 7, 7
routes.loc['C1', 'E'], routes.loc['E', 'C1'] = 8, 8
routes.loc['C1', 'F'], routes.loc['F', 'C1'] = 10, 10
routes.loc['D1', 'E'], routes.loc['E', 'D1'] = 9, 9
routes.loc['D1', 'F'], routes.loc['F', 'D1'] = 6, 6
routes

,A,B,C,D,C1,D1,E,F
A,999,999,5,6,999,999,999,999
B,999,999,4,7,999,999,999,999
C,5,4,999,999,999,999,999,999
D,6,7,999,999,999,999,999,999
C1,999,999,999,999,999,999,8,10
D1,999,999,999,999,999,999,9,6
E,999,999,999,999,8,9,999,999
F,999,999,999,999,10,6,999,999


Variables

In [437]:
num_personnel = 5
num_alpha_v = 1
num_beta_v = 1

alpha_p, beta_p = {}, {}
for n in range(num_personnel):
    for a in range(num_alpha_v):
        for h in points_holding_2:
            for d in points_destination:
                alpha_p[(n, a, h, d)] = model.NewBoolVar(f'x_{n}{a}{h}{d}')
for n in range(num_personnel):
    for b in range(num_beta_v):
        for s in points_start:
            for h in points_holding_1:
                beta_p[(n, b, s, h)] = model.NewBoolVar(f'y_{n}{b}{s}{h}')

alpha_v, beta_v = {}, {}
for h in points_holding_2:
    for d in points_destination:
        alpha_v[(h, d)] = model.NewBoolVar(f'p_{h}{d}')
        alpha_v[(d, h)] = model.NewBoolVar(f'p_{d}{h}')
for s in points_start:
    for h in points_holding_1:
        beta_v[(s, h)] = model.NewBoolVar(f'q_{s}{h}')
        beta_v[(h, s)] = model.NewBoolVar(f'q_{h}{s}')

node_p = {}
for n in range(num_personnel):
    for i in points_start + points_holding_1 + points_holding_2 + points_destination:
        node_p[(n, i)] = model.NewBoolVar(f'z_{n}{i}')

node_v = {}
for i in points:
    node_v[i] = model.NewBoolVar(f'w_{i}')

departure, arrival = {}, {}
for i in points:
    departure[i] = model.NewIntVar(0, 10000, f'd_{i}')
    arrival[i] = model.NewIntVar(0, 10000, f'a_{i}')

Constraints

In [438]:
vehicle_capacity = 3
loading_unloading_time = 5

## Vehicle arrives and departs from same node if node is visited
for i in points_holding_2:
    arrive_at_holding = [alpha_v[var] for var in alpha_v if var[1]==i]
    depart_from_holding = [alpha_v[var] for var in alpha_v if var[0]==i]
    model.Add(sum(arrive_at_holding) == node_v[i])
    model.Add(sum(depart_from_holding) == node_v[i])
for i in points_holding_1:
    arrive_at_holding = [beta_v[var] for var in beta_v if var[1]==i]
    depart_from_holding = [beta_v[var] for var in beta_v if var[0]==i]
    model.Add(sum(arrive_at_holding) == node_v[i])
    model.Add(sum(depart_from_holding) == node_v[i])
for i in points_destination:
    arrive_at_destination = [alpha_v[var] for var in alpha_v if var[1]==i]
    depart_from_destination = [alpha_v[var] for var in alpha_v if var[0]==i]
    model.Add(sum(arrive_at_destination) == node_v[i])
    model.Add(sum(depart_from_destination) == node_v[i])
for i in points_start:
    arrive_at_start = [beta_v[var] for var in beta_v if var[1]==i]
    depart_from_start = [beta_v[var] for var in beta_v if var[0]==i]
    model.Add(sum(arrive_at_start) == node_v[i])
    model.Add(sum(depart_from_start) == node_v[i])

## Personnel arrives and departs from same holding
for i, j in zip(points_holding_1, points_holding_2):
    for n in range(num_personnel):
        arrive_at_holding = [beta_p[var] for var in beta_p if (var[0]==n and var[3]==i)]
        depart_from_holding = [alpha_p[var] for var in alpha_p if (var[0]==n and var[2]==j)]
        model.Add(sum(arrive_at_holding) == node_p[n, i])
        model.Add(sum(depart_from_holding) == node_p[n, i])

## Personnel visits node only if it boards the vehicle
for n in range(num_personnel):
    for j in points_destination:
        arrive_at_node = [var for var in alpha_p if (var[0]==n and var[3]==j)]
        total = 0
        for var in arrive_at_node:
            xp = model.NewIntVar(0, 10000, 'xp')   ## temporary variable for nonlinear constraint
            model.AddMultiplicationEquality(xp, alpha_p[var], alpha_v[var[-2:]])
            total += xp
        model.Add(node_p[n, j] == total)
for n in range(num_personnel):
    for j in points_holding_1:
        arrive_at_node = [var for var in beta_p if (var[0]==n and var[3]==j)]
        total = 0
        for var in arrive_at_node:
            yq = model.NewIntVar(0, 10000, 'yq')
            model.AddMultiplicationEquality(yq, beta_p[var], beta_v[var[-2:]])
            total += yq
        model.Add(node_p[n, j] == total)

## Personnel takes route only if vehicle takes route
for n in range(num_personnel):
    route_by_n = [var for var in alpha_p if var[0]==n]
    for var in route_by_n:
        model.Add(alpha_p[var] <= alpha_v[var[-2:]])
for n in range(num_personnel):
    route_by_n = [var for var in beta_p if var[0]==n]
    for var in route_by_n:
        model.Add(beta_p[var] <= beta_v[var[-2:]])

## No personnel going backwards in direction
for i in points_destination:
    depart_from_destination = [alpha_p[var] for var in alpha_p if var[2]==i]
    model.Add(sum(depart_from_destination) == 0)
for i in points_holding_1:
    depart_from_holding = [alpha_p[var] for var in alpha_p if ((var[2]==i) and (var[3] in points_start))]
    model.Add(sum(depart_from_holding) == 0)

## All personnel must end up at destination
for n in range(num_personnel):
    arrive_at_destination = [node_p[var] for var in node_p if ((var[0]==n) and (var[1] in points_destination))]
    model.Add(sum(arrive_at_destination) == 1)

## Vehicle capacity constraints
for i in points_start:
    for j in points_holding_1:
        arrive_at_holding = [var for var in beta_p if (var[2]==i and var[3]==j)]
        total = 0
        for var in arrive_at_holding:
            yq = model.NewIntVar(0, 10000, 'yq')   ## temporary variable for nonlinear constraint
            model.AddMultiplicationEquality(yq, beta_p[var], beta_v[var[2:]])
            total += yq
        model.Add(total <= vehicle_capacity)
for i in points_holding_2:
    for j in points_destination:
        arrive_at_destination = [var for var in alpha_p if (var[2]==i and var[3]==j)]
        total = 0
        for var in arrive_at_destination:
            xp = model.NewIntVar(0, 10000, 'xp')   ## temporary variable for nonlinear constraint
            model.AddMultiplicationEquality(xp, alpha_p[var], alpha_v[var[2:]])
            total += xp
        model.Add(total <= vehicle_capacity)

## Departure time after arrival time
for i in points:
    if i not in ['A', 'F']:
        model.Add(departure[i] >= arrival[i] + loading_unloading_time)

## Arrival time at node = departure time + travel time
for j in points_destination:
    arrive_at_node = [var for var in alpha_v if var[1]==j]
    total = 0
    for var in arrive_at_node:
        pd = model.NewIntVar(0, 10000, 'pd')   ## temporary variable for nonlinear constraint
        model.AddMultiplicationEquality(pd, alpha_v[var], departure[var[0]])
        total += pd + alpha_v[var] * routes.loc[var]
    model.Add(arrival[j] == total)
for j in points_holding_1:  ## beta
    arrive_at_node = [var for var in beta_v if var[1]==j]
    total = 0
    for var in arrive_at_node:
        qd = model.NewIntVar(0, 10000, 'qd')
        model.AddMultiplicationEquality(qd, beta_v[var], departure[var[0]])
        total += qd + beta_v[var] * routes.loc[var]
    model.Add(arrival[j] == total)
for j in points_holding_2:  ## alpha
    arrive_at_node = [var for var in alpha_v if var[1]==j]
    total = 0
    for var in arrive_at_node:
        qd = model.NewIntVar(0, 10000, 'qd')
        model.AddMultiplicationEquality(qd, alpha_v[var], departure[var[0]])
        total += qd + alpha_v[var] * routes.loc[var]
    model.Add(arrival[j] == total)
for j in points_start:
    arrive_at_node = [var for var in beta_v if var[1]==j]
    total = 0
    for var in arrive_at_node:
        qd = model.NewIntVar(0, 10000, 'qd')
        model.AddMultiplicationEquality(qd, beta_v[var], departure[var[0]])
        total += qd + beta_v[var] * routes.loc[var]
    model.Add(arrival[j] == total)

## Subtour elimination constraints
# for q in beta_v:
#     model.Add(arrival[q[1]] >= arrival[q[0]] - 100*(1-beta_v[q]))
# for q in alpha_v:
#     model.Add(arrival[q[1]] >= arrival[q[0]] - 100*(1-alpha_v[q]))

## No subtour (btw 2 nodes only)
for i in points_start:
    for j in points_holding_1:
        model.Add(beta_v[i, j] + beta_v[j, i] <= 1)
for i in points_holding_2:
    for j in points_destination:
        model.Add(alpha_v[i, j] + alpha_v[j, i] <= 1)

## All nodes are visited once only
for w in node_v:
    model.Add(node_v[w] == 1)

## If personnel at C, personnel will be at C1
for n in range(num_personnel):
    for i, j in zip(points_holding_1, points_holding_2):
        model.Add(node_p[n, i] == node_p[n, j])

Initial state

In [439]:
# model.Add(alpha_v[('B', 'D')] == 1)
# model.Add(beta_p[(1, 0, 'A', 'C')] == 1)

model.Add(departure['A'] == 0)
model.Add(node_p[(0, 'A')] == 1)
model.Add(node_p[(1, 'A')] == 1)
model.Add(node_p[(2, 'B')] == 1)
model.Add(node_p[(3, 'B')] == 1)
model.Add(node_p[(4, 'B')] == 1)

Objective function

In [440]:
model.Minimize(sum([arrival[i] for i in points_destination])) # since departure is 0, just minimise max arrival time?
# model.Minimize(max(list(arrival.values())) - max(list(departure.values()))) # cannot find max/min of decision variables
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('objective value:', solver.ObjectiveValue())
elif status == cp_model.INFEASIBLE:
    print("No solution found")
else:
    print("Something is wrong, check the status and the log of the solve")

objective value: 68.0


Results

In [441]:
# print("\nnode_v")
# for k, v in node_v.items():
#     print(k, solver.Value(v))

print("\nnode_p")
test = pandas.DataFrame(index=range(5), columns=['A', 'B', 'C', 'D'])
for i in node_p:
    test.loc[i[0], i[1]] = solver.Value(node_p[i])
print(test)

print("\narrival & departure")
for k, v in zip(arrival.items(), departure.items()):
    print(k[0], solver.Value(k[1]), solver.Value(v[1]))

print("\nalpha_v")
for k, v in alpha_v.items():
    print(k, solver.Value(v))

print("\nbeta_v")
for k, v in beta_v.items():
    print(k, solver.Value(v))

print("\nalpha_p")
for k, v in alpha_p.items():
    print(k, solver.Value(v))

print("\nbeta_p")
for k, v in beta_p.items():
    print(k, solver.Value(v))


node_p
   A  B  C  D   C1   D1    E    F
0  1  0  0  1  0.0  1.0  1.0  0.0
1  1  0  0  1  0.0  1.0  1.0  0.0
2  0  1  1  0  1.0  0.0  0.0  1.0
3  0  1  1  0  1.0  0.0  0.0  1.0
4  0  1  1  0  1.0  0.0  0.0  1.0

arrival & departure
A 37 0
B 18 23
C 27 32
D 6 11
C1 33 38
D1 6 11
E 20 25
F 48 0

alpha_v
('C1', 'E') 0
('E', 'C1') 1
('C1', 'F') 1
('F', 'C1') 0
('D1', 'E') 1
('E', 'D1') 0
('D1', 'F') 0
('F', 'D1') 1

beta_v
('A', 'C') 0
('C', 'A') 1
('A', 'D') 1
('D', 'A') 0
('B', 'C') 1
('C', 'B') 0
('B', 'D') 0
('D', 'B') 1

alpha_p
(0, 0, 'C1', 'E') 0
(0, 0, 'C1', 'F') 0
(0, 0, 'D1', 'E') 1
(0, 0, 'D1', 'F') 0
(1, 0, 'C1', 'E') 0
(1, 0, 'C1', 'F') 0
(1, 0, 'D1', 'E') 1
(1, 0, 'D1', 'F') 0
(2, 0, 'C1', 'E') 0
(2, 0, 'C1', 'F') 1
(2, 0, 'D1', 'E') 0
(2, 0, 'D1', 'F') 0
(3, 0, 'C1', 'E') 0
(3, 0, 'C1', 'F') 1
(3, 0, 'D1', 'E') 0
(3, 0, 'D1', 'F') 0
(4, 0, 'C1', 'E') 0
(4, 0, 'C1', 'F') 1
(4, 0, 'D1', 'E') 0
(4, 0, 'D1', 'F') 0

beta_p
(0, 0, 'A', 'C') 0
(0, 0, 'A', 'D') 1
(0, 0, 'B', 'C') 

Visualise

In [442]:
print("\nbeta_v")

beta_v_list = [k for k, v in beta_v.items() if solver.Value(v)]
beta_v_dict = {k[0]: k for k in beta_v_list}
beta_v_route = []
beta_v_start = 'A'
while True:
    try:
        sublist = beta_v_dict[beta_v_start]
    except KeyError:
        break
    beta_v_route.append(sublist)
    del beta_v_dict[beta_v_start]
    beta_v_start = sublist[-1]

beta_v_route


beta_v


[('A', 'D'), ('D', 'B'), ('B', 'C'), ('C', 'A')]

In [443]:
print("\nalpha_v")

alpha_v_list = [k for k, v in alpha_v.items() if solver.Value(v)]
alpha_v_dict = {k[1]: k for k in alpha_v_list}
alpha_v_route = []
alpha_v_end = 'F'
while True:
    try:
        sublist = alpha_v_dict[alpha_v_end]
    except KeyError:
        break
    alpha_v_route.insert(0, sublist)
    del alpha_v_dict[alpha_v_end]
    alpha_v_end = sublist[0]

alpha_v_route


alpha_v


[('F', 'D1'), ('D1', 'E'), ('E', 'C1'), ('C1', 'F')]